In [1]:
from helper import *

pd.set_option('display.max_rows', None)

In [2]:
# load preds
preds_2025 = pd.read_csv('./data/clean/predictions_2025.csv')

# load fantasy data
fantasy_2024 = pd.read_csv('./data/clean/pff_fantasy_data.csv').query('Year == 2024')[['Player', 'PPGPosRank_half-ppr', 'grades_offense']]

# add 'PPGPosRank_half-ppr' as 'rank_2024' col in preds_2025
preds_2025 = preds_2025.merge(fantasy_2024, how='left', left_on='player', right_on='Player').drop(columns=['Player'])
preds_2025 = preds_2025.rename(columns={'PPGPosRank_half-ppr': 'rank_2024', 'grades_offense': 'grade_2024'})

In [8]:
pos = 'TE'
num_players = 36
preds_2025.query('pos == @pos').sort_values('rank_2024', ascending=True).head(50)

,player,team,pos,games_played_pct_pred,ppg_pred,adj_pred,pos_rank,rank_2024,grade_2024
60,George Kittle,SFO,TE,0.783755,10.625777,8.328003,2.0,1,92.1
25,Brock Bowers,LVR,TE,0.829698,12.470600,10.346828,1.0,2,85.1
68,Trey McBride,ARI,TE,0.825982,9.688758,8.002739,3.0,3,86.8
140,Taysom Hill,NOR,TE,0.722614,6.451742,4.662117,13.0,4,71.8
105,Jonnu Smith,MIA,TE,0.822520,7.176982,5.903215,7.0,5,78.2
115,David Njoku,CLE,TE,0.747640,7.198138,5.381617,8.0,6,64.0
96,Mark Andrews,BAL,TE,0.779636,8.239586,6.423878,6.0,7,83.8
78,Travis Kelce,KAN,TE,0.802557,9.469428,7.599758,4.0,8,70.9
90,Sam LaPorta,DET,TE,0.802692,8.389908,6.734509,5.0,9,73.8
122,Dallas Goedert,PHI,TE,0.751795,6.861399,5.158364,10.0,10,65.7


In [13]:
teams_2024 = pd.read_csv('./data/pff/team/2024.csv')
teams_2024['blocking grade'] = (teams_2024['Pass Blocking Grade'] + teams_2024['Run Blocking Grade']).div(2)
teams_2024 = teams_2024[['Tm', 'Points For', 'Points Against', 'blocking grade', 'Pass Blocking Grade', 'Run Blocking Grade']].copy().sort_values('blocking grade', ascending=False).reset_index(drop=True)
teams_2024

,Tm,Points For,Points Against,blocking grade,Pass Blocking Grade,Run Blocking Grade
0,Atlanta Falcons,389,423,77.80,72.8,82.8
1,San Francisco 49ers,389,436,76.00,72.0,80.0
2,Denver Broncos,425,311,75.65,83.6,67.7
3,Indianapolis Colts,377,427,74.75,72.5,77.0
4,Philadelphia Eagles,463,303,74.55,73.9,75.2
5,Tampa Bay Buccaneers,502,385,74.10,82.5,65.7
6,Detroit Lions,564,342,73.40,68.4,78.4
7,Kansas City Chiefs,385,326,72.15,70.7,73.6
8,Green Bay Packers,460,338,72.10,82.1,62.1
9,Arizona Cardinals,400,379,71.20,75.8,66.6


In [15]:
teams_2024.sort_values('Points For', ascending=False)

,Tm,Points For,Points Against,blocking grade,Pass Blocking Grade,Run Blocking Grade
6,Detroit Lions,564,342,73.40,68.4,78.4
11,Buffalo Bills,525,368,69.60,74.7,64.5
13,Baltimore Ravens,518,361,68.40,73.3,63.5
5,Tampa Bay Buccaneers,502,385,74.10,82.5,65.7
23,Washington Commanders,485,391,61.15,64.5,57.8
29,Cincinnati Bengals,472,434,53.50,56.3,50.7
4,Philadelphia Eagles,463,303,74.55,73.9,75.2
8,Green Bay Packers,460,338,72.10,82.1,62.1
16,Minnesota Vikings,432,332,67.60,69.3,65.9
2,Denver Broncos,425,311,75.65,83.6,67.7
